In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib
import numpy as np
import itertools
import scipy
import pandas as pd
import seaborn as sns
from scipy import stats
import os
import nept

from loading_data import get_data
from analyze_tuning_curves import get_only_tuning_curves
from analyze_decode_bytrial import decode_trial
from analyze_decode import get_decoded_zones
from utils_maze import find_zones, get_trials, get_zones, get_trial_idx

In [ ]:
import info.r063d2 as info

In [ ]:
events, position, spikes, _, _ = get_data(info)
xedge, yedge = nept.get_xyedges(position, binsize=8)

In [ ]:
len(xedge), len(yedge)

In [ ]:
phase = "phase3"
t_start = info.task_times[phase].start
t_stop = info.task_times[phase].stop

sliced_pos = position.time_slice(t_start, t_stop)

feeder1_times = []
for feeder1 in events['feeder1']:
    if t_start < feeder1 < t_stop:
        feeder1_times.append(feeder1)

feeder2_times = []
for feeder2 in events['feeder2']:
    if t_start < feeder2 < t_stop:
        feeder2_times.append(feeder2)

path_pos = get_zones(info, sliced_pos)

trials_idx, trial_epochs = get_trial_idx(path_pos['u'].time, 
                                         path_pos['shortcut'].time, 
                                         path_pos['novel'].time,
                                         feeder1_times, 
                                         feeder2_times, 
                                         t_stop)

shortcut_epochs = [trial_epochs[idx] for idx in trials_idx["shortcut"]]
u_epochs = [trial_epochs[idx] for idx in trials_idx["u"]]
novel_epochs = [trial_epochs[idx] for idx in trials_idx["novel"]]

In [ ]:
plt.plot(position.x, position.y, ".")
plt.show()

In [ ]:
binned_maze_shape = (len(yedge)-1, len(xedge)-1)

In [ ]:
xx, yy = np.meshgrid(xedge, yedge)

In [ ]:
u_pos = np.zeros(binned_maze_shape).astype(bool)

u_position = position.time_slice(info.task_times["phase1"].start, info.task_times["phase1"].stop)
u_occupancy = nept.get_occupancy(u_position, yedge, xedge)
u_pos[u_occupancy > 0.] = True

u_position = position.time_slice(info.task_times["phase2"].start, info.task_times["phase2"].stop)
u_occupancy = nept.get_occupancy(u_position, yedge, xedge)
u_pos[u_occupancy > 0.] = True

In [ ]:
plt.figure()
pp = plt.pcolormesh(xx, yy, u_pos, cmap="Greys")
colourbar = plt.colorbar(pp)
plt.tight_layout()

In [ ]:
filtered = scipy.signal.convolve2d(u_pos, kernel, mode='same')

In [ ]:
plt.figure()
pp = plt.pcolormesh(xx, yy, u_area, cmap="Greys")
colourbar = plt.colorbar(pp)
plt.tight_layout()

In [ ]:
u_area =  np.zeros(binned_maze_shape).astype(bool)
filtered = scipy.signal.convolve2d(u_pos, kernel, mode='same')
u_area[filtered>0] = True

In [ ]:

filtered = scipy.signal.convolve2d(shortcut_pos, kernel, mode='same')
shortcut_area[filtered>0] = True

In [ ]:
novel_area =  np.zeros(binned_maze_shape).astype(bool)
filtered = scipy.signal.convolve2d(novel_pos, kernel, mode='same')
novel_area[filtered>0] = True

In [ ]:
plt.figure()
pp = plt.pcolormesh(xx, yy, novel_area, cmap="Greys")
colourbar = plt.colorbar(pp)
plt.tight_layout()

In [ ]:
shortcut_area =  np.zeros(binned_maze_shape).astype(bool)
shortcut_pos = np.zeros(binned_maze_shape).astype(bool)

shortcut_position = position.time_slice(info.task_times["phase3"].start, info.task_times["phase3"].stop)
shortcut_occupancy = nept.get_occupancy(shortcut_position, yedge, xedge)
shortcut_pos[(shortcut_occupancy > 0.1) & (~u_area)] = True
filtered = scipy.signal.convolve2d(shortcut_pos, kernel, mode='same')
shortcut_pos[filtered<=1] = False
filtered = scipy.signal.convolve2d(shortcut_pos, kernel, mode='same')
shortcut_area[filtered>0] = True

In [ ]:
plt.figure()
pp = plt.pcolormesh(xx, yy, shortcut_pos, cmap="Greys")
colourbar = plt.colorbar(pp)
plt.tight_layout()

In [ ]:
kernel = [[0,1,0],[1,1,1],[0,1,0]]

In [ ]:
shortcut_pos = np.zeros(binned_maze_shape).astype(bool)
for epoch in shortcut_epochs:
    shortcut_position = position.time_slice(epoch.start, epoch.stop)
    shortcut_occupancy = nept.get_occupancy(shortcut_position, yedge, xedge)
    shortcut_pos[(shortcut_occupancy > 0.04) & (~u_pos)] = True
filtered = scipy.signal.convolve2d(shortcut_pos, kernel, mode='same')
shortcut_pos[filtered<=1] = False

In [ ]:
novel_pos = np.zeros(binned_maze_shape).astype(bool)
for epoch in novel_epochs:
    novel_position = position.time_slice(epoch.start, epoch.stop)
    novel_occupancy = nept.get_occupancy(novel_position, yedge, xedge)
    novel_pos[(novel_occupancy > 0.04) & (~doesthiswork) & (~shortcut_pos)] = True
filtered = scipy.signal.convolve2d(novel_pos, kernel, mode='same')
novel_pos[filtered<=1] = False

In [ ]:
plt.figure()
pp = plt.pcolormesh(xx, yy, novel_pos, cmap="Greys")
colourbar = plt.colorbar(pp)
plt.tight_layout()

In [ ]:
plt.figure()
plt.pcolormesh(xx, yy, u_pos + shortcut_pos + novel_pos, cmap="Greys")
colourbar = plt.colorbar(pp)
plt.tight_layout()